In [1]:
import numpy as np 
import torch 
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [3]:
bikes_numpy=np.loadtxt(


    '../data/p1ch4/bike-sharing-dataset/hour-fixed.csv',dtype=np.float32,
    delimiter=",",skiprows=1,converters={1:lambda x:float(x[8:10])}

)
bikes=torch.from_numpy(bikes_numpy)
bikes


tensor([[1.00e+00, 1.00e+00,  ..., 1.30e+01, 1.60e+01],
        [2.00e+00, 1.00e+00,  ..., 3.20e+01, 4.00e+01],
        ...,
        [1.74e+04, 3.10e+01,  ..., 4.80e+01, 6.10e+01],
        [1.74e+04, 3.10e+01,  ..., 3.70e+01, 4.90e+01]])

In [4]:
bikes.shape,bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [5]:
#view

daily_bikes=bikes.view(-1,24,bikes.shape[1])
daily_bikes.shape ,daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [6]:
daily_bikes=daily_bikes.transpose(1,2)
daily_bikes.shape,daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [7]:
first_day=bikes[:24].long()
weather_onehot=torch.zeros(first_day.shape[0],4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

In [9]:
weather_onehot.scatter_(

    dim=1,
    index=first_day[:,9].unsqueeze(1).long(),
    value=1.0
)

tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]])

In [11]:
weather_onehot.scatter_(

    dim=1,
    index=first_day[:,9].unsqueeze(1).long()-1 ,
    value=1.0
)

tensor([[1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [0., 1., 1., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.]])

In [12]:
torch.cat((bikes[:24],weather_onehot),1)[:1]

tensor([[ 1.00,  1.00,  1.00,  0.00,  1.00,  0.00,  0.00,  6.00,  0.00,
          1.00,  0.24,  0.29,  0.81,  0.00,  3.00, 13.00, 16.00,  1.00,
          1.00,  0.00,  0.00]])

In [13]:
daily_weather_onehot=torch.zeros(

    daily_bikes.shape[0],4,
    daily_bikes.shape[2]
)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [14]:
daily_weather_onehot.scatter_(

    1,daily_bikes[:,9,:].long().unsqueeze(1)-1,1.0
)
daily_weather_onehot.shape 

torch.Size([730, 4, 24])

In [15]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [16]:
daily_bikes[:,9,:]=(daily_bikes[:,9,:]-1.0) /3.0

In [17]:
temp=daily_bikes[:,10,:]
temp_min=torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [ ]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))